In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json
import glob
import random
import collections
from glob import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import seaborn as sns
import shutil
import keras
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import keras
import json
import tensorflow as tf 
from keras.layers import Input
from keras import Sequential
from keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16

In [2]:
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    tn = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1-y_true) * (y_pred), 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1-y_pred), 0, 1)))
    

    f1_val = tp / ( tp + ( (1/2) * (fp+fn) ) + K.epsilon())
    return f1_val

In [18]:
from glob import glob
import cv2
from PIL import Image, ImageStat
import math
trn1='D:/Inv_Data_Imbalance/newdata/To Baylor 2022-02-03/To Baylor 2022-02-03/Davis Dam Preserved Sample 2020-07-24/2022-01-29_6th_Vel_2c_No_NR/Veligers/*/'
trn2='D:/Inv_Data_Imbalance/newdata/To Baylor 2022-02-03/To Baylor 2022-02-03/Davis Dam Preserved Sample 2020-07-24/2022-01-29_6th_Vel_2c_No_NR/Non-Veligers/*/'
tr1= glob(trn1)
tr2= glob(trn2)

In [25]:
from glob import glob
import cv2
from PIL import Image, ImageStat
import math
trn1='D:/Inv_Data_Imbalance/newdata/To Baylor 2022-02-03/To Baylor 2022-02-03/Davis Dam Preserved Sample 2020-07-24/2022-01-29_6th_Vel_3b_No_NR/Veligers/*/'
trn2='D:/Inv_Data_Imbalance/newdata/To Baylor 2022-02-03/To Baylor 2022-02-03/Davis Dam Preserved Sample 2020-07-24/2022-01-29_6th_Vel_3b_No_NR/Non-Veligers/*/'
tr1= glob(trn1)
tr2= glob(trn2)

In [17]:
from glob import glob
import cv2
from PIL import Image, ImageStat
import math
trn1='D:/Inv_Data_Imbalance/newdata/To Baylor 2022-02-04/To Baylor 2022-02-04/Davis Dam Preserved Sample 2020-07-24/2022-01-29_6th_Vel_1f_No_NR/Veligers/*/'
trn2='D:/Inv_Data_Imbalance/newdata/To Baylor 2022-02-04/To Baylor 2022-02-04/Davis Dam Preserved Sample 2020-07-24/2022-01-29_6th_Vel_1f_No_NR/Non-Veligers/*/'
tr1= glob(trn1)
tr2= glob(trn2)

In [17]:
len(tr2)

115

In [26]:
tran_index_inv = np.round( len(tr1)* .99  )
tran_index_noninv = np.round( len(tr2)* .99  )
tran_index_noninv

273.0

In [27]:
data = []
label = []
breath = []
total = 0

for j in tr1[:(int) (tran_index_inv)]:
    label.append(1)
    a = glob(j+'/*')
    breath.append(len(a))
    total = total + len(a)
    
for j in tr2[:(int) (tran_index_noninv)]:
    label.append(0)
    a = glob(j+'/*')
    breath.append(len(a)) 
    total = total + len(a)

for j in range(0,len(tr1[:(int) (tran_index_inv)])):
    a = glob(tr1[j]+'/*')
    for k in range(0,6):
        data.append(a[k])

for j in range(0,len(tr2[:(int) (tran_index_noninv)])):
    a = glob(tr2[j]+'/*')
    for k in range(0,6):
        data.append(a[k])        
        

imgdata=[]
for i in range(len(data)):
    a = Image.open(data[i])
    b = a.resize((15, 15))
    c = np.array(b)
    imgdata.append(c.reshape(15,15,3))
    
from tensorflow.keras.utils import to_categorical
idata = np.array(imgdata)
X_test = idata
X_test = X_test.astype('float32') / 255.
X_test = np.reshape(X_test, (len(X_test),15,15,3))


In [28]:
end= 0
test_df= []
breath = 6

i = 0
for i in range(0, len(label)):
    deff = []
    for k in range(0, (breath)):
        
        index = (i*6+k)
        
        deff.append(X_test[index])
        
    test_df.append(deff)
    
Y_test = to_categorical(label)
test_df = np.array(test_df)
test_df,Y_test = shuffle(test_df,Y_test)
np.shape(test_df)

(323, 6, 15, 15, 3)

In [29]:
model = load_model('models/RNN_model1.h5') 

In [30]:
Y_test = Y_test
pred = model.predict(test_df)
p = np.round(pred)
f1 = get_f1(Y_test, p)
f1

y_p = []
for i in range(len(p)):
    if ( p[i][0] == 0 ):
        y_p.append(1)
    else :
        y_p.append(0)
y_p = np.array(y_p)
y_t = []
for i in range(len(Y_test)):
    if ( Y_test[i][0] == 0 ):
        y_t.append(1)
    else :
        y_t.append(0)
y_t = np.array(y_t)

from sklearn.metrics import confusion_matrix
a=(confusion_matrix(y_t, y_p , labels=[0,1]))
a.T

array([[234,  17],
       [ 39,  33]], dtype=int64)

In [31]:
f1

<tf.Tensor: shape=(), dtype=float32, numpy=0.8266254>